#  US Gap Analysis Project - WV Breeding Bird Atlas Data Comparison 
Nathan Tarr and Jessie Jordan

## Elevation
We investigated the agreement between WV Breeding Bird Atlas (2011-2015) and USGS Gap Analysis Project data on the elevational limits in occurrence.

We compared three things: the elevations of survey locations where individuals were recorded by the WVBBA, the maximum and minimum elevations of habitat grid cells as mapped by GAP for each species, and the elevation parameter values from GAP habitat models.  We obtained WVBBA data from WV biologists who ran that project.  We determined the maximum and minimum elevations of habitat grid cells by summarizing their elevations with the arcpy package ("map_max" and "map_min").  Habitat model parameters were retrieved from ScienceBase items for the models.  The border between GAP's southeastern and northeastern modeling regions passes through WV, so habitat maps within WV could potentially have been produced in part from different sets of parameters.  We therefore used southeastern and northeastern model parameters in our comparisons ("NE_max", "NE_min", "SE_max", and "SE_min"), which, in some cases, complicated comparisons.

To quantify the agreement between data sets, we tallied the number of individuals that the WVBBA recorded above and below the max elevations from habitat maps ("individuals_over(n)" and "individuals_under(n)"), as well as the differences between maximum and minimum elevations from WVBBA surveys and GAP maps ("max_diff" and "min_diff").  

In [1]:
import pandas as pd, repo_functions as fun, numpy as np
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 15)

# Elevation summary csv
eDF0 = pd.read_csv(fun.resultsDir + "elevation_summary.csv", header=0, names=["GAP_code", "WVBBA_code", "common_name",
                                                                              "map_max", "map_min", "SE_max", "SE_min", 
                                                                              "NE_max", "NE_min", "WVBBA_max", "WVBBA_min",
                                                                              "max_diff(%)", "min_diff(%)", "scientific_name",
                                                                              "max_diff", "min_diff", "WVBBA_individuals(n)", 
                                                                              "individuals_over(n)", "individuals_under(n)"])
print("A sample of values\n")
eDF0.drop(["min_diff(%)","max_diff(%)"], axis=1, inplace=True)
eDF1 = eDF0.copy()
eDF0.set_index("GAP_code", inplace=True)
print(eDF0.head().T)

A sample of values

GAP_code                          bcogrx               bcoyex             bhowrx                 brbnux           brevix
WVBBA_code                          COGR                 COYE               HOWR                   RBNU             REVI
common_name               Common Grackle  Common Yellowthroat         House Wren  Red-breasted Nuthatch   Red-eyed Vireo
map_max                             1370                 1482               1480                   1482             1477
map_min                               75                   75                 77                     86               77
SE_max                              1371                  NaN                NaN                   2500              NaN
SE_min                                 0                  NaN                NaN                   1000              NaN
NE_max                              1371                  NaN                NaN                   2500              NaN
NE_min      

A complication existed regarding the elevations of WVBBA survey data that necessitated specifying an allowable difference between elevation values below which values are considered equal.  That is because observers used a point count protocol in which bird detections were attributed to the survey point.  Although the survey location was recorded as a point, the area surveyed was actually a polygon approximating a circle.  Thus, birds could have been detected from within a range of elevations despite being attributed to a point.  

The range of elevations surveyed in a point count is determined by the width of the circle that was effectively surveyed and the terrain around the point.  The distance from the observer's location (i.e. the point) at which individuals could have been detected, which ultimately determines the width of the circle, is affected by site characteristics including vegetation structure.  Thus, the elevation ranges assocated with detections varies among points.  If the survey area was large and the terrain steep, then the range of elevations at which birds could have been detected is large.  Without knowing the conditions at each survey location, it is impossible to precisely attribute each record to an elevation, but ignoring this issue would introduce error in our assessment, therefore we approximated an allowable margin of error of 75m for elevation values that we used when comparing WVBBA records to GAP data: values were considered equal when they were within 75m of each other.  We chose 75m because individuals can often be heard from distance of 100m or more in auditory bird surveys and WV, in general, is mountainous.

In [2]:
# Set an allowable margin of error for elevation (m)
elev_error = 75

### Which models have innapropriate GAP elevation parameters that should be adjusted for WV?
Cases where the elevation parameter should be adjusted can be identified by determining which species were detected in significant frequency at elevations above the maximum or below the minimum elevation predicted with the habitat map *and* for which a map elevation extreme equals the corresponding elevation parameter in the model.  Equivalent elevation limits from the maps and models indicate that a model's elevation limits restricted model output (the map).  It is possible for the elevation parameter to differ from the map extremes when other model parameters, such as cover type selections, restrict output to elevations that are higher than the minimum or lower than the maximum.  Defining a "significant frequency" of WVBBA occurrences that defy the model parameter is subjective, but we used a value of 1% of total detections.  Additionally, we applied the margin of error discussed above so that at least one detection had to be more than 75m above the GAP maximum or below the minimum in order for frequency of errors to be assessed. 

In [3]:
# Which species were detected above the maximum elevation from habitat maps?
df0 = eDF1[eDF1["individuals_over(n)"] > 0]
overMax = df0.filter(items=['GAP_code', 'common_name', 'individuals_over(n)', 'WVBBA_max', 'map_max', 'max_diff', 'SE_max', 
                            'NE_max', 'WVBBA_individuals(n)'], axis=1)
 
# Omit records with difference greater than the set "elev_error".
overMax = overMax[overMax['max_diff'] > elev_error]

# Filter out species w/ <1% of individuals conflicting with the model
overMax['individuals_over(%)'] = 100*(overMax['individuals_over(n)']/overMax['WVBBA_individuals(n)'])
overMax.sort_values(by="individuals_over(n)", ascending=False, inplace=True)
overMax = overMax[overMax['individuals_over(%)'] > 1]

# For which species was a model responsible and needs to be adjusted? If one of the model max's is NULL, then the map output 
# max wasn't necessarily constrained by it, so exclude it.  
overMax2 = overMax.copy().dropna()
overMax2.drop(["WVBBA_individuals(n)"], axis=1, inplace=True)
overMax2.set_index(["GAP_code"], inplace=True)
print("Maximum elevation needs to be adjusted in the models for these species")
print(overMax2.filter(["common_name", "individuals_over(n)", "individuals_over(%)", "max_diff", "WVBBA_max", "map_max", 
              "SE_max", "NE_max"], axis=1))

Maximum elevation needs to be adjusted in the models for these species
                     common_name  individuals_over(n)  individuals_over(%)  max_diff  WVBBA_max  map_max  SE_max  NE_max
GAP_code                                                                                                                
bchspx          Chipping Sparrow                  136            16.229117       387       1300      913   914.0   914.0
brbwox    Red-bellied Woodpecker                   26             3.576341       361       1260      899   900.0   900.0
bwiflx         Willow Flycatcher                   16            16.494845       371       1220      849   850.0   850.0
bororx            Orchard Oriole                   13            10.569106       331        940      609   610.0   610.0
bnoflx          Northern Flicker                    8             1.731602       122       1340     1218  1219.0  1219.0
bbekix         Belted Kingfisher                    8             9.302326       2

In [4]:
# Which species were detected below min elevation from habitat maps?  Comments from the cell above are analogous here. 
df1 = eDF1[eDF1["individuals_under(n)"] > 0]
overMin = df1.filter(items=['GAP_code', 'common_name', 'individuals_under(n)', 'WVBBA_min', 'map_min', 'min_diff', 'SE_min', 
                            'NE_min', 'WVBBA_individuals(n)'], axis=1)
overMin = overMin[overMin['min_diff'] > elev_error]
overMin['individuals_under(%)'] = 100*(overMin['individuals_under(n)']/overMin['WVBBA_individuals(n)'])
overMin.sort_values(by="individuals_under(n)", ascending=False, inplace=True)
overMin = overMin[overMin['individuals_under(%)'] > 1]
overMin2 = overMin.copy().dropna()
overMin2.set_index(["GAP_code"], inplace=True)
print("\n\nMinimum elevation needs to be adjusted in the models for these species")
print(overMin2.filter(["common_name", "individuals_under(n)", "individuals_under(%)", "min_diff", "WVBBA_min", "map_min", 
                       "SE_min", "NE_min"], axis=1))



Minimum elevation needs to be adjusted in the models for these species
                common_name  individuals_under(n)  individuals_under(%)  min_diff  WVBBA_min  map_min  SE_min  NE_min
GAP_code                                                                                                             
bbhvix    Blue-headed Vireo                    24              4.897959        81        220      301   300.0   300.0


### For which species did something other than an elevation parameter restrict output?
Significant numbers of WVBBA detections above the GAP map maximum elevation or below the minimum indicated that something other than the model parameter restricted the habitat map to erroneously restricted elevations in cases where the map maximum or minimum did not equal the elevation limits in the models.

In [5]:
overMax3 = overMax[overMax['GAP_code'].isin(overMax2.index) == False].copy()
overMax3.drop(["WVBBA_individuals(n)"], axis=1, inplace=True)
overMax3.set_index("GAP_code", inplace=True)
print("\n\nSomething other than a maximum elevation parameter could be erroneously restricting map output to low elevations.")
print(overMax3.filter(["common_name", "individuals_over(n)", "individuals_over(%)", "max_diff", "WVBBA_max", "map_max", 
                       "SE_max", "NE_max"], axis=1))



Something other than a maximum elevation parameter could be erroneously restricting map output to low elevations.
             common_name  individuals_over(n)  individuals_over(%)  max_diff  WVBBA_max  map_max  SE_max  NE_max
GAP_code                                                                                                        
beaphx    Eastern Phoebe                  202            38.403042       714       1160      446     NaN     NaN


In [6]:
overMin3 = overMin[overMin['GAP_code'].isin(overMin2.index) == False].copy()
overMin3.drop(["WVBBA_individuals(n)"], axis=1, inplace=True)
overMin3.set_index("GAP_code", inplace=True)
print("\n\nSomething other than a minimum elevation parameter could be erroneously restricting map output to high elevations.")
print(overMin3.filter(["common_name", "individuals_under(n)", "individuals_under(%)", "min_diff", "WVBBA_min", "map_min", 
                       "SE_min", "NE_min"], axis=1))



Something other than a minimum elevation parameter could be erroneously restricting map output to high elevations.
              common_name  individuals_under(n)  individuals_under(%)  min_diff  WVBBA_min  map_min  SE_min  NE_min
GAP_code                                                                                                           
bdejux    Dark-eyed Junco                    12               4.83871       141        560      701   700.0     NaN


### For which species is the elevation parameter obsolete in WV?
Comparing the GAP habitat models to the habitat maps reveals cases where the elevation model parameters are not affecting the model output within WV.

In [7]:
# For which species is maximum map elevation less than the maximum elevation parameter
misMax = eDF0.copy().filter(items=['GAP_code', 'common_name', 'map_max', 'SE_max', 'NE_max'], axis=1).dropna()
misMax2 = misMax[(misMax['map_max'] + 1 < misMax['SE_max']) & (misMax['map_max'] + 1 < misMax['NE_max'])]
print("These species have obsolete maximum elevations for WV")
print(misMax2)

# For which species is minimum map elevation less than the minimum elevation parameter
misMin = eDF0.copy().filter(items=['GAP_code', 'common_name', 'map_min', 'SE_min', 'NE_min'], axis=1).dropna()
misMin2 = misMin[(misMin['map_min'] - 1 > misMin['SE_min']) & (misMin['map_min'] - 1 > misMin['NE_min'])]
print("\n\nThese species have obsolete minimum elevations for WV")
print(misMin2)

These species have obsolete maximum elevations for WV
                       common_name  map_max  SE_max  NE_max
GAP_code                                                   
brbnux       Red-breasted Nuthatch     1482  2500.0  2500.0
bcachx          Carolina Chickadee     1476  1850.0  1850.0
bgckix      Golden-crowned Kinglet     1482  2500.0  3450.0
bybsax    Yellow-bellied Sapsucker     1481  2500.0  2000.0
bbrthx              Brown Thrasher     1475  1676.0  1676.0
btutix             Tufted Titmouse     1482  1524.0  1524.0
bwavix              Warbling Vireo     1476  3200.0  3200.0
bwiwrx                 Winter Wren     1482  2500.0  2500.0
bbadox                  Barred Owl     1477  1500.0  1500.0
bbobox                    Bobolink     1178  2500.0  2500.0
brbgrx      Rose-breasted Grosbeak     1482  2500.0  2500.0
bbhvix           Blue-headed Vireo     1482  2500.0  2500.0
bwbnux     White-breasted Nuthatch     1481  1524.0  1524.0
bdejux             Dark-eyed Junco     1482  2

## Conclusions
It was possible to identify ways to improve the GAP habitat models with data from the WVBBA.  Additionally, merely comparing the GAP data to itself (i.e. model parameters to the output maps) revealed some areas for improvement.  The results above identify those specific changes that should be made to models.  However, they also reveal an aspect of the modeling paradigm that limits model quality and iterative improvement.  Using the six large, modeling regions means that models cannot be parameterized or refined locally even when knowledge is available to warrant it. Modelers have to assign parameters that can accomodate all the environments where a species could occur throughout the modeling region, even if tolerable environments exist along a gradients. In the case of elevation, an obsolete parameter within WV may be justified due to suitable environments in ME and any changes made to models for WV will affect habitat predictions from ME to FL, LA, and AR.  Additionally, the habitat associations (e.g., elevational limits) of species may vary across such broad regions.  In order to  incorporate state-level data into model improvements without reducing their specificity, it will be necessary to accomodate smaller and/or customized modeling regions.